<a href="https://colab.research.google.com/github/Xintong120/Front-End-Interview/blob/master/02_CLIP%E5%90%91%E9%87%8F%E5%8C%96_%E7%9F%A5%E8%AF%86%E5%BA%93%E6%9E%84%E5%BB%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 环境准备和依赖安装

In [14]:
# 安装必要的依赖
!pip install transformers torch torchvision
!pip install pillow numpy
!pip install tqdm loguru
!pip install faiss-cpu  # 可选：用于高效向量搜索

In [42]:
import os
import json
import pickle
import numpy as np
from pathlib import Path
from tqdm import tqdm
from loguru import logger
from typing import List,Dict,Any,Optional
import pandas as pd

import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import base64
from io import BytesIO


In [16]:
# Google Drive 挂载 (仅在 Colab 环境需要)
from google.colab import drive
drive.mount('/content/drive')

# 设置 Drive 路径
drive_root = Path('/content/drive/MyDrive')
input_dir = drive_root / 'RAG_处理结果'  # 第一阶段的输出目录

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. 配置路径和参数

In [17]:
# 配置路径 - Colab 版本
BASE_DIR = input_dir
INPUT_JSON_PATH = BASE_DIR / "all_pdf_page_chunks.json"  # 注意文件名
OUTPUT_DIR = drive_root / "knowledge_base"
FINAL_KB_PATH = OUTPUT_DIR / "multimodal_knowledge_base.pkl"
CHUNKS_JSON_PATH = OUTPUT_DIR / "processed_chunks.json"

# 创建输出目录
OUTPUT_DIR.mkdir(exist_ok=True)

# CLIP模型配置
CLIP_MODEL_NAME = "openai/clip-vit-base-patch32"  # ~600MB，免费使用
BATCH_SIZE = 16  # 根据GPU内存调整
MAX_TEXT_LENGTH = 77  # CLIP文本输入限制

print(f"输入JSON文件: {INPUT_JSON_PATH}")
print(f"输出目录: {OUTPUT_DIR}")
print(f"最终知识库: {FINAL_KB_PATH}")
print(f"CLIP模型: {CLIP_MODEL_NAME}")

输入JSON文件: /content/drive/MyDrive/RAG_处理结果/all_pdf_page_chunks.json
输出目录: /content/drive/MyDrive/knowledge_base
最终知识库: /content/drive/MyDrive/knowledge_base/multimodal_knowledge_base.pkl
CLIP模型: openai/clip-vit-base-patch32


# 3. 加载和初始化CLIP模型

In [18]:
# 加载CLIP模型和处理器
print("正在加载CLIP模型和处理器...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

try:
  clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
  clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
  clip_model.eval()
  print("CLIP模型和处理器加载成功！")
except Exception as e:
  print(f"加载CLIP模型和处理器时出错: {e}")
  raise

正在加载CLIP模型和处理器...
使用设备: cpu


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

CLIP模型和处理器加载成功！


# 4. 数据加载和预处理

In [24]:
# 加载阶段一的JSON数据
print("正在加载阶段一的JSON数据...")
with open(INPUT_JSON_PATH, 'r', encoding='utf-8') as f:
    raw_content_list = json.load(f)
print(f"加载完成，共 {len(raw_content_list)} 个内容块")

# 数据结构分析
type_counts = {}
for item in raw_content_list:
    item_type = item.get('type', 'unknown')
    type_counts[item_type] = type_counts.get(item_type, 0) + 1

print("\n📊 内容类型统计:")
for item_type, count in type_counts.items():
    print(f"  {item_type}: {count} 个")

正在加载阶段一的JSON数据...
加载完成，共 4294 个内容块

📊 内容类型统计:
  unknown: 4294 个


## 清洗和结构化原始数据，生成标准化的chunk格式

In [27]:
# 检查原始数据的结构
print("检查前几个数据项的结构:")
for i in range(min(3, len(raw_content_list))):
    print(f"\n第{i+1}个数据项:")
    print(f"  类型: {type(raw_content_list[i])}")
    if isinstance(raw_content_list[i], dict):
        print(f"  键: {list(raw_content_list[i].keys())}")
        for key, value in raw_content_list[i].items():
            if isinstance(value, str) and len(value) > 100:
                print(f"  {key}: {value[:100]}...")
            else:
                print(f"  {key}: {value}")
    else:
        print(f"  内容: {raw_content_list[i]}")

检查前几个数据项的结构:

第1个数据项:
  类型: <class 'dict'>
  键: ['id', 'content', 'metadata']
  id: 伊利股份-大象起舞龙头远航-2020072627页_page_0
  content: 请仔细阅读本报告末页声明 
证券研究报告 | 首次覆盖报告 
2020 年07 月26 日 
伊利股份（600887.SH） 
大象起舞，龙头远航 
看历史：步步为营，终成乳业龙头。公司创始于1993...
  metadata: {'filename': '伊利股份-大象起舞龙头远航-2020072627页', 'page': 0, 'source': 'pymupdf', 'total_pages': 27}

第2个数据项:
  类型: <class 'dict'>
  键: ['id', 'content', 'metadata']
  id: 伊利股份-大象起舞龙头远航-2020072627页_page_1
  content: 2020 年07 月26 日 
P.2                                   请仔细阅读本报告末页声明 
 
财务报表和主要财务比率 
  
资产负债表（百万元）  
 ...
  metadata: {'filename': '伊利股份-大象起舞龙头远航-2020072627页', 'page': 1, 'source': 'pymupdf', 'total_pages': 27}

第3个数据项:
  类型: <class 'dict'>
  键: ['id', 'content', 'metadata']
  id: 伊利股份-大象起舞龙头远航-2020072627页_page_2
  content: qRtMoOoRpQmOuMqPqMtMsRaQ8QbRoMrRmOoOfQqQtMeRnPoR6MqQzQMYoNoOMYmQrO
 2020 年07 月26 日 
P.3             ...
  metadata: {'filename': '伊利股份-大象起舞龙头远航-2020072627页', 'page': 2, 'source': 'pymupdf', 'total_pages': 27}


In [30]:
def clean_and_structure_data(raw_data: List[Dict]) -> List[Dict]:
    """
    清洗和结构化原始数据，生成标准化的chunk格式
    """
    processed_chunks = []

    for i, item in enumerate(tqdm(raw_data, desc="数据清洗")):
        try:
            # 提取基本信息
            content_text = item.get('content', '')
            item_id = item.get('id', f'chunk_{i:06d}')
            metadata = item.get('metadata', {})

            #从metadata中提取信息
            source_file = metadata.get('filename','unknow')
            page_num = metadata.get('page', 0)


            # 处理不同类型的内容
            chunk = {
                'id': item_id,
                'content': content_text.strip() if content_text else '',
                'type': 'text',
                'metadata': {
                    'source_file': source_file,
                    'page_num': page_num,
                    'original_index': i,
                    'total_pages': metadata.get('total_pages', 0)
                }
            }


            # 只保留有内容的chunk
            if chunk['content'].strip():
                processed_chunks.append(chunk)

        except Exception as e:
            logger.warning(f"处理第{i}个内容块时出错: {e}")
            continue

    return processed_chunks

In [31]:
# 执行数据清洗
processed_chunks = clean_and_structure_data(raw_content_list)
print(f"\n✅ 数据清洗完成，有效chunk数: {len(processed_chunks)}")

数据清洗: 100%|██████████| 4294/4294 [00:00<00:00, 273617.75it/s]


✅ 数据清洗完成，有效chunk数: 4294


# 5. CLIP向量化函数

In [32]:
def encode_text_with_clip(texts: List[str], batch_size: int = BATCH_SIZE) -> np.ndarray:
    """
    使用CLIP对文本进行批量编码
    """
    all_embeddings = []

    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="文本向量化"):
            batch_texts = texts[i:i + batch_size]

            # 截断过长的文本
            truncated_texts = [text[:500] for text in batch_texts]  # 保守截断

            try:
                # CLIP文本编码
                inputs = clip_processor(text=truncated_texts, return_tensors="pt",
                                      padding=True, truncation=True, max_length=MAX_TEXT_LENGTH)
                inputs = {k: v.to(device) for k, v in inputs.items()}

                text_features = clip_model.get_text_features(**inputs)
                text_features = text_features / text_features.norm(dim=-1, keepdim=True)  # 归一化

                all_embeddings.append(text_features.cpu().numpy())

            except Exception as e:
                logger.error(f"文本编码出错: {e}")
                # 使用零向量作为fallback
                fallback_embedding = np.zeros((len(batch_texts), 512))
                all_embeddings.append(fallback_embedding)

    return np.vstack(all_embeddings)

def encode_image_with_clip(image_data: str) -> Optional[np.ndarray]:
    """
    使用CLIP对单个图像进行编码
    """
    try:
        # 解码base64图像
        if image_data.startswith('data:image'):
            image_data = image_data.split(',')[1]

        image_bytes = base64.b64decode(image_data)
        image = Image.open(BytesIO(image_bytes)).convert('RGB')

        with torch.no_grad():
            inputs = clip_processor(images=image, return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}

            image_features = clip_model.get_image_features(**inputs)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)

            return image_features.cpu().numpy().flatten()

    except Exception as e:
        logger.error(f"图像编码出错: {e}")
        return None

print("✅ CLIP向量化函数定义完成")

✅ CLIP向量化函数定义完成


# 6. 批量向量化处理

In [33]:
# 分离文本和图像内容
text_chunks = []
image_chunks = []

for chunk in processed_chunks:
    if chunk.get('has_image', False):
        image_chunks.append(chunk)
    else:
        text_chunks.append(chunk)

print(f"📝 纯文本chunk: {len(text_chunks)} 个")
print(f"🖼️ 包含图像chunk: {len(image_chunks)} 个")
print(f"📊 总计: {len(processed_chunks)} 个")

📝 纯文本chunk: 4294 个
🖼️ 包含图像chunk: 0 个
📊 总计: 4294 个


In [34]:
# 处理纯文本chunk
print("\n🔄 开始处理纯文本内容...")
if text_chunks:
    text_contents = [chunk['content'] for chunk in text_chunks]
    text_embeddings = encode_text_with_clip(text_contents)

    # 将向量添加到chunk中
    for i, chunk in enumerate(text_chunks):
        chunk['vector'] = text_embeddings[i].tolist()
        chunk['vector_type'] = 'text'

    print(f"✅ 文本向量化完成: {len(text_chunks)} 个")
else:
    print("⚠️ 没有纯文本内容需要处理")


🔄 开始处理纯文本内容...


文本向量化: 100%|██████████| 269/269 [08:40<00:00,  1.93s/it]


✅ 文本向量化完成: 4294 个


In [36]:
# 处理图像chunk
print("\n🔄 开始处理图像内容...")
successful_images = 0
failed_images = 0

for chunk in tqdm(image_chunks, desc="图像向量化"):
    image_data = chunk.get('image_data')
    if image_data:
        image_vector = encode_image_with_clip(image_data)
        if image_vector is not None:
            chunk['vector'] = image_vector.tolist()
            chunk['vector_type'] = 'image'
            successful_images += 1
        else:
            # 图像处理失败，使用文本向量作为fallback
            text_vector = encode_text_with_clip([chunk['content']])
            chunk['vector'] = text_vector[0].tolist()
            chunk['vector_type'] = 'text_fallback'
            failed_images += 1
    else:
        # 没有图像数据，使用文本向量
        text_vector = encode_text_with_clip([chunk['content']])
        chunk['vector'] = text_vector[0].tolist()
        chunk['vector_type'] = 'text_fallback'
        failed_images += 1

print(f"✅ 图像向量化完成:")
print(f"  成功处理图像: {successful_images} 个")
print(f"  使用文本fallback: {failed_images} 个")


🔄 开始处理图像内容...


图像向量化: 0it [00:00, ?it/s]

✅ 图像向量化完成:
  成功处理图像: 0 个
  使用文本fallback: 0 个


# 7. 构建最终知识库

In [38]:
# 合并所有处理后的chunk
final_chunks = text_chunks + image_chunks
# 验证所有chunk都有向量
chunks_with_vectors = [chunk for chunk in final_chunks if 'vector' in chunk]
print(f"\n📊 最终统计:")
print(f"  总chunk数: {len(final_chunks)}")
print(f"  有向量的chunk: {len(chunks_with_vectors)}")
print(f"  向量维度: {len(chunks_with_vectors[0]['vector']) if chunks_with_vectors else 'N/A'}")

# 统计向量类型
vector_type_counts = {}
for chunk in chunks_with_vectors:
    vtype = chunk.get('vector_type', 'unknown')
    vector_type_counts[vtype] = vector_type_counts.get(vtype, 0) + 1

print("\n🏷️ 向量类型统计:")
for vtype, count in vector_type_counts.items():
    print(f"  {vtype}: {count} 个")


📊 最终统计:
  总chunk数: 4294
  有向量的chunk: 4294
  向量维度: 512

🏷️ 向量类型统计:
  text: 4294 个


In [43]:
# 构建知识库数据结构
knowledge_base = {
    'metadata': {
        'version': '1.0',
        'created_at': str(pd.Timestamp.now()),
        'clip_model': CLIP_MODEL_NAME,
        'total_chunks': len(chunks_with_vectors),
        'vector_dimension': len(chunks_with_vectors[0]['vector']) if chunks_with_vectors else 0,
        'vector_type_counts': vector_type_counts
    },
    'chunks': chunks_with_vectors
}

print(f"\n🏗️ 知识库构建完成")
print(f"  元数据: {len(knowledge_base['metadata'])} 项")
print(f"  数据块: {len(knowledge_base['chunks'])} 个")


🏗️ 知识库构建完成
  元数据: 6 项
  数据块: 4294 个


# 8. 持久化保存

In [44]:
# 保存为JSON格式（便于查看和调试）
print("💾 保存JSON格式...")
with open(CHUNKS_JSON_PATH, 'w', encoding='utf-8') as f:
    json.dump(knowledge_base, f, ensure_ascii=False, indent=2)

json_size = CHUNKS_JSON_PATH.stat().st_size / 1024 / 1024
print(f"✅ JSON文件保存完成: {CHUNKS_JSON_PATH} ({json_size:.2f} MB)")

# 保存为Pickle格式（更高效的加载）
print("💾 保存Pickle格式...")
with open(FINAL_KB_PATH, 'wb') as f:
    pickle.dump(knowledge_base, f)

pickle_size = FINAL_KB_PATH.stat().st_size / 1024 / 1024
print(f"✅ Pickle文件保存完成: {FINAL_KB_PATH} ({pickle_size:.2f} MB)")

print(f"\n📁 知识库文件:")
print(f"  JSON格式: {CHUNKS_JSON_PATH} ({json_size:.2f} MB)")
print(f"  Pickle格式: {FINAL_KB_PATH} ({pickle_size:.2f} MB)")

💾 保存JSON格式...
✅ JSON文件保存完成: /content/drive/MyDrive/knowledge_base/processed_chunks.json (75.29 MB)
💾 保存Pickle格式...
✅ Pickle文件保存完成: /content/drive/MyDrive/knowledge_base/multimodal_knowledge_base.pkl (29.48 MB)

📁 知识库文件:
  JSON格式: /content/drive/MyDrive/knowledge_base/processed_chunks.json (75.29 MB)
  Pickle格式: /content/drive/MyDrive/knowledge_base/multimodal_knowledge_base.pkl (29.48 MB)


9. **知识库质量验证**

In [45]:
# 简单的向量搜索测试
def cosine_similarity(v1, v2):
    """计算余弦相似度"""
    v1, v2 = np.array(v1), np.array(v2)
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

def simple_search(query_text: str, top_k: int = 3):
    """简单的向量搜索测试"""
    # 对查询进行向量化
    query_vector = encode_text_with_clip([query_text])[0]

    # 计算与所有chunk的相似度
    similarities = []
    for chunk in chunks_with_vectors:
        sim = cosine_similarity(query_vector, chunk['vector'])
        similarities.append((sim, chunk))

    # 排序并返回top_k
    similarities.sort(key=lambda x: x[0], reverse=True)
    return similarities[:top_k]

# 测试搜索功能
test_queries = [
    "财务报表",
    "营业收入",
    "资产负债表"
]

print("\n🔍 知识库搜索测试:")
for query in test_queries:
    print(f"\n查询: '{query}'")
    results = simple_search(query, top_k=2)
    for i, (score, chunk) in enumerate(results, 1):
        content_preview = chunk['content'][:100] + "..." if len(chunk['content']) > 100 else chunk['content']
        print(f"  {i}. 相似度: {score:.3f} | {content_preview}")

print("\n✅ 搜索测试完成！")


🔍 知识库搜索测试:

查询: '财务报表'


文本向量化: 100%|██████████| 1/1 [00:00<00:00, 10.26it/s]


  1. 相似度: 0.959 | 证券研究报告
  2. 相似度: 0.955 | 证券研究报告
附表：财务预测与估值
资产负债表（百万元）
2023
2024
2025E
2026E
2027E
利润表（百万元）
2023
2024
2025E
2026E
2027E
现金及现金等...

查询: '营业收入'


文本向量化: 100%|██████████| 1/1 [00:00<00:00, 12.84it/s]


  1. 相似度: 0.966 | 一、
营销中心：负责市场营销业务，进行市场开拓、销售业务、品牌推广及产品管理等工作，由营销副总经理负责管理。营销中心下设重客部、经
销商客户部、渠道拓展部、行销部及产品管理部等。公司对每类大B客户都...
  2. 相似度: 0.962 | -53-
附录：财务预测表

查询: '资产负债表'


文本向量化: 100%|██████████| 1/1 [00:00<00:00, 13.14it/s]


  1. 相似度: 0.973 | 证券研究报告
  2. 相似度: 0.965 | 1 投资逻辑 
市场担心速冻米面行业的竞争会加剧。我们认为：连锁餐饮供应体系相对封闭，进入壁垒
较高，公司具有先发优势。千味央厨对餐饮场景的理解深刻，公司始终深入一线，研究餐
饮应用场景的痛点和需求。...

✅ 搜索测试完成！


# 10. 生成部署配置文件

In [46]:
# 生成部署所需的配置信息
deployment_config = {
    "knowledge_base_path": str(FINAL_KB_PATH.name),  # 相对路径
    "clip_model_name": CLIP_MODEL_NAME,
    "vector_dimension": len(chunks_with_vectors[0]['vector']) if chunks_with_vectors else 512,
    "total_chunks": len(chunks_with_vectors),
    "deployment_ready": True,
    "required_packages": [
        "transformers",
        "torch",
        "numpy",
        "gradio",
        "pillow"
    ]
}

config_path = OUTPUT_DIR / "deployment_config.json"
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(deployment_config, f, ensure_ascii=False, indent=2)

print(f"\n⚙️ 部署配置文件已生成: {config_path}")
print("\n🎉 阶段二完成！知识库已构建完成，可以进入阶段三进行检索调试。")
print("\n📋 下一步:")
print("  1. 运行阶段三Notebook进行检索逻辑调试")
print("  2. 完成后即可部署到HuggingFace Spaces")
print(f"  3. 记得将 {FINAL_KB_PATH.name} 文件上传到Spaces项目中")


⚙️ 部署配置文件已生成: /content/drive/MyDrive/knowledge_base/deployment_config.json

🎉 阶段二完成！知识库已构建完成，可以进入阶段三进行检索调试。

📋 下一步:
  1. 运行阶段三Notebook进行检索逻辑调试
  2. 完成后即可部署到HuggingFace Spaces
  3. 记得将 multimodal_knowledge_base.pkl 文件上传到Spaces项目中
